## `pyvo.mivot` package demonstrator

The code below shows how the model views provided by the  `pyvo.mivot` package can be used.

- Both code and data are  mostly taken out if the `pyvo.mivot` test suite. 

It aims at showing how the model views provided by the package can be used.

- This examples are based on the initual version of the `pyvo.mivot` package.
- The code is totaly model agnostic
- There is no connection with Astropy classes

The process is 3 steps:
1. The MIVOT block (XML) is read by the viewer and references to table columns and GLOBALS objects are then resolved
2. The XMl block is tranformed in a dictionary where dm roles are sued as keys.
3. This dictionary is used to buikd a dynamic Python object that ca be used as any other object.
    1. The keys od the dictionary are transformed in object attributes.
    2. Dynamic object have no class. Instances can be modified but new instance cannot be created bu by doing copies.

This notebook has been develop before pyvo 1.6 has been published: works with the devlopement code

In [ ]:
%pwd
%cd ../../pyvo
%pwd
!pip install -e .
%cd ../dm-usecases/notebooks
%pwd

`pyvo.mivot`is a new pyvo feature, it must be explicetly activated.

In [ ]:
from pyvo.mivot import MivotViewer
from pyvo.utils import activate_features

activate_features('MIVOT')

## Connect the MIVOT viewer with the VOTable.

The first data row is read and interpreted and the iterator is rewound.

In [ ]:
votable = "../pyvo-ci-sample/gaia_epoch_propagation_full.xml"
mivot_viewer = MivotViewer(votable)

## Get an XML serialization of the mapped model

When the viewer is created it reafs the fierst data row to intialize the model view.
1. The MIVOT block (XML) is read and references are resolved
    1. referenced objects are copied at the right places
    2. values of ATTRIBUTE with @ref are set with the FIELD values
2. Some XML attributes are added to facilitate further processing (or development)

The XML output, pretty printed here, can be used as such with e.g. XPath queries.

In [ ]:
from pyvo.mivot.utils.xml_utils import XmlUtils
XmlUtils.pretty_print(mivot_viewer.xml_view)

## Get a mivot instance

The code below gets the MIVOT instance generated by the viewer. 

- Note that the data table can be iterated by the viewer in the same way as Astropy.
- Each time a new row is reasd, the MIVOT instance is updated with the new values.
- It recommended to work with a reference on the MIVOT instance for performance reasons.

The MIVOT instance a representation method that provides a pretty serialization of the dictionary that has been used to build the instance.

In [ ]:
mivot_object = mivot_viewer.dm_instance
while mivot_viewer.next():
    print(mivot_object)
    break


## Access model fields through the MIVOT instance

Mapped quantity can be accessed through the paths defined by the model.

To be Python-compliant, path elements that contain ":" or "." are escaped with "_".

Each model leaf has 3 attributes:
1. The value
2. The unit as a string, not an Astropy unit yet
3. The identificator of the column (if it exists) the value comes from

In [ ]:
with MivotViewer(votable) as mivot_viewer:
    mivot_object = mivot_viewer.dm_instance
    space_frame = mivot_instance.EpochPosition_coordSys.PhysicalCoordSys_frame.spaceRefFrame.value
    while mivot_viewer.next():
        ra = mivot_object.longitude.value
        ra_unit = mivot_object.longitude.unit
        dec = mivot_object.latitude.value
        pmra = mivot_object.pmLongitude.value
        pmra_unit = mivot_object.pmLongitude.unit

        pmdec = mivot_object.pmLatitude.value
        epoch =  mivot_object.epoch.value
        print(f"Year {epoch}: position({space_frame}) = [{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")


## Run the viewer from de pyvo DAL Service

The viewer can be directly initialized from a DAL service response.

In the cell below, the query has been tuned to return one row.

In [22]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from pyvo.dal.scs import SCSService

vizier_url = "https://cdsarc.cds.unistra.fr/beta/viz-bin/mivotconesearch/I/239/hip_main"
scs_srv = SCSService(vizier_url)
mivot_viewer = MivotViewer(
    scs_srv.search(
        pos=SkyCoord(ra=52.26708 * u.degree, dec=59.94027 * u.degree, frame="icrs"),
        radius=0.05,
    )
)
mivot_object = mivot_viewer.dm_instance
while mivot_viewer.next():
    frame = mivot_object.Coordinate_coordSys.spaceRefFrame.value
    ra = mivot_object.longitude.value
    ra_unit = mivot_object.longitude.unit
    dec = mivot_object.latitude.value
    pmra = mivot_object.pmLongitude.value
    pmra_unit = mivot_object.pmLongitude.unit

    pmdec = mivot_object.pmLatitude.value
    epoch =  mivot_object.epoch.value
    print(f"Year {epoch}: position({frame})=[{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")

Year 1991.25: position(ICRS)=[52.26722684 59.94033461 deg] proper motion = [-0.82 -1.85 mas/yr]


## Apply the model view on data rows read with Astropy

Once the mivot viewe has been initialized in a regular way, the VOTable data can be iterated with Astropy and the MIVOT viewer can be invoked to provide a model view on any record. 

In [23]:
from astropy.io.votable import parse

parsed_votable = parse(votable)
table = parsed_votable.resources[0].tables[0]

mivot_viewer = MivotViewer(votable)

mivot_instance = mivot_viewer.dm_instance
dmtype = mivot_instance.dmtype = "EpochPosition"
space_frame = mivot_instance.EpochPosition_coordSys.PhysicalCoordSys_frame.spaceRefFrame.value
for rec in table.array:
    mivot_instance.update(rec)
    ra = mivot_instance.longitude.value
    ra_unit = mivot_instance.longitude.unit
    dec = mivot_instance.latitude.value
    pmra = mivot_instance.pmLongitude.value
    pmra_unit = mivot_instance.pmLongitude.unit

    pmdec = mivot_instance.pmLatitude.value
    epoch =  mivot_instance.epoch.value
    print(f"Year {epoch}: position({space_frame}) = [{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")


Year 2016.5: position(ICRS) = [307.79115807079 20.43108005561 deg] proper motion = [-2.557 -5.482 mas/yr]
Year 2016.5: position(ICRS) = [307.79582391363 20.43253083107 deg] proper motion = [-2.78 -3.853 mas/yr]
Year 2016.5: position(ICRS) = [307.79737366047 20.43558827154 deg] proper motion = [None None mas/yr]
Year 2016.5: position(ICRS) = [307.78856137441 20.43011713943 deg] proper motion = [-2.781 -3.61 mas/yr]
Year 2016.5: position(ICRS) = [307.78977228741 20.43150439876 deg] proper motion = [-1.975 -3.712 mas/yr]
